# Preprocessing dataset

### Prerequisites

Download raw datasets via `git lfs fetch --include "data/raw/*.csv"`

In [1]:
!pip install pandas
!pip install beautifulsoup4
!pip install sentence-transformers
!pip install numpy
!pip install pandas

In [2]:
import os
import os.path as path
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import HTML
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

In [3]:
root_dir = path.dirname(path.abspath(os.getcwd()))
data_dir = path.join(root_dir, 'data')
raw_data_dir = path.join(data_dir, 'raw')
embeddings_dir = path.join(data_dir, 'embeddings')

os.makedirs(embeddings_dir, exist_ok=True)

raw_vacancies_path = path.join(raw_data_dir, 'hhparser_vacancy.csv')
raw_cvs_path = path.join(raw_data_dir, 'dst-3.0_16_1_hh_database.csv')

vacancies_embeddings_path = path.join(embeddings_dir, 'vacancies.csv')
cvs_embeddings_path = path.join(embeddings_dir, 'cvs.csv')

raw_vacancies_path

'/Users/alv.popov/prj/DL-CASE2/data/raw/hhparser_vacancy.csv'

In [4]:
model = SentenceTransformer('cointegrated/LaBSE-en-ru')

In [13]:
n_batch = 1000

def chunks(n, dataframe):
    for i in range(0, len(dataframe), n):
        print(i)
        yield i, dataframe.iloc[i: i+n]

def to_csv(filename, dataframe):
    print(f"Total items: {len(dataframe)}")
    with open(filename, 'w') as file:
        file.write(f'{dataframe.index.name},embeddings')
        for i, chunk in chunks(n_batch, dataframe):
            try:
                embeddings = model.encode(chunk['text'].tolist())
                str_embeddings = map(lambda x: f'"{":".join(map(str, x))}"', embeddings)
                content = '\n' + '\n'.join(
                    map(
                        lambda x: ','.join(x),
                        zip(map(str, dataframe.index), str_embeddings)
                    )
                )
                file.write(content)
            except BaseException as e:
                print(f"Error in {i} chunk: {e}")

## Vacancies

In [6]:
raw_vacancies = pd.read_csv(raw_vacancies_path, index_col='id')
raw_vacancies

,name,has_test,response_letter_required,salary_from,salary_to,salary_currency,salary_gross,published_at,created_at,parsed_at,url,employer_name,description,alternate_url,area_id,area_name
id,,,,,,,,,,,,,,,,
38228194,Project R’n’D / Технолог R’n’D,False,True,NaN,NaN,NaN,NaN,2020-07-26 23:40:42+00,2020-07-26 23:40:42+00,2020-07-27 03:03:03.154874+00,https://api.hh.ru/vacancies/38228194?host=hh.ru,Масл фектори,<p><strong>MF Kitchen</strong> — группа компан...,https://hh.ru/vacancy/38228194,1,Москва
36911636,"Продавец-кассир (Москва, Богданова, 16)",False,False,33500.0,40000.0,RUR,True,2020-07-27 00:28:12+00,2020-07-27 00:28:12+00,2020-07-27 03:03:04.258759+00,https://api.hh.ru/vacancies/36911636?host=hh.ru,Пятёрочка,<p><strong>Обязанности:</strong></p><ul><li>Об...,https://hh.ru/vacancy/36911636,1,Москва
36909000,"Продавец-кассир (Москва, Новогиреевская, 11/36)",False,False,33500.0,38000.0,RUR,True,2020-08-24 00:16:48+00,2020-08-24 00:16:48+00,2020-08-24 02:59:43.766137+00,https://api.hh.ru/vacancies/36909000?host=hh.ru,Пятёрочка,<p><strong>Обязанности:</strong></p><ul><li>Об...,https://hh.ru/vacancy/36909000,1,Москва
38985954,"Product owner бизнес–стрима ""Аналитика и Отчет...",False,False,NaN,NaN,NaN,NaN,2020-09-08 18:29:25+00,2020-09-08 18:29:25+00,2020-09-08 19:02:47.261747+00,https://api.hh.ru/vacancies/38985954?host=hh.ru,Банк ВТБ (ПАО),<p>Наша позиция предполагает участие в програм...,https://hh.ru/vacancy/38985954,1,Москва
40108800,Business Development Director (CTO),False,False,NaN,NaN,NaN,NaN,2020-11-25 08:56:45+00,2020-11-25 08:56:45+00,2020-11-28 06:59:37.872153+00,https://api.hh.ru/vacancies/40108800?host=hh.ru,Студия Олега Чулакова,<p>Студия Олега Чулакова — это дизайн-студия №...,https://hh.ru/vacancy/40108800,1,Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38834939,Оператор 1C,False,False,48000.0,NaN,RUR,True,2020-08-26 06:52:49+00,2020-08-26 06:52:49+00,2020-08-26 07:02:58.340756+00,https://api.hh.ru/vacancies/38834939?host=hh.ru,ТД ЛФБ,"<p> </p> <p>ТД «ЛФБ» - семейная компания, веде...",https://hh.ru/vacancy/38834939,1,Москва
37345373,Стажер Управления финансирования торговли,False,False,40000.0,40000.0,RUR,True,2020-07-25 13:23:01+00,2020-07-25 13:23:01+00,2020-07-28 11:01:18.939989+00,https://api.hh.ru/vacancies/37345373?host=hh.ru,UniCredit Bank,<p>КРАТКАЯ ИНФОРМАЦИЯ:</p> <p>Отдел финансиров...,https://hh.ru/vacancy/37345373,1,Москва
38602820,Преподаватель математики/информатики,False,False,NaN,NaN,NaN,NaN,2020-08-13 16:30:47+00,2020-08-13 16:30:47+00,2020-08-15 07:01:49.548725+00,https://api.hh.ru/vacancies/38602820?host=hh.ru,Центр дополнительного образования Новый Взгляд,<p><strong>Обязанности:</strong></p> <p> </p> ...,https://hh.ru/vacancy/38602820,1,Москва


In [7]:
HTML(raw_vacancies.iloc[0]['description'])

In [8]:
def process_description(description):
    description = str(description)
    description = BeautifulSoup(description, features="html.parser").get_text()
    return description

raw_vacancies['text'] = (
    raw_vacancies['name'] + " . " +
    raw_vacancies['description'].apply(process_description) + " . " +
    raw_vacancies['area_name']
).astype(str)
raw_vacancies.iloc[0]['text']

/var/folders/3y/lpnlj_f962vbckl7626fst540000gq/T/ipykernel_73010/1388809492.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, features="html.parser").get_text()


'Project R’n’D / Технолог R’n’D . MF Kitchen — группа компаний, занимающаяся развитием единой платформы, цель которой — забрать вопрос еды на себя и помочь нашим клиентам заниматься своими любимыми делами, освободив себя от походов по магазинам, готовки и мытья посуды. Платформа поделена на несколько направлений, в каждом из которых независимые продуктовые команды с взаимными интеграциями. А именно: MF — e-com платформа, объединяющая в себе су-бренды Kitchen, Cook и Chef- сервисы по подписке на разноплановое питание. Trend by MF- розничная сеть кафе быстрого здорового питания, объединяющая офлайн и онлайн продажи и коммуникации с клиентом. Если ты веришь, что здоровое питание меняет мир к лучшему, хочешь стремительного развития в команде таких же профессионалов и не боишься нетривиальных задач — возможно, нам с тобой по пути.   Обязанности: - Контролировать и координировать проекты по разработке новых блюд в соответствии с концепцией и требованиями фабрики-кухни, развивать существующие

In [9]:
processed_vacancies = raw_vacancies[raw_vacancies['text'].notnull()][['text']]
processed_vacancies

,text
id,
38228194,Project R’n’D / Технолог R’n’D . MF Kitchen — ...
36911636,"Продавец-кассир (Москва, Богданова, 16) . Обяз..."
36909000,"Продавец-кассир (Москва, Новогиреевская, 11/36..."
38985954,"Product owner бизнес–стрима ""Аналитика и Отчет..."
40108800,Business Development Director (CTO) . Студия О...
...,...
38834939,"Оператор 1C . ТД «ЛФБ» - семейная компания, ..."
37345373,Стажер Управления финансирования торговли . КР...
38602820,Преподаватель математики/информатики . Обязанн...


In [15]:
to_csv(vacancies_embeddings_path, processed_vacancies.sample(20_000))

Total items: 20000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


## CVs

In [16]:
raw_cvs = pd.read_csv(raw_cvs_path, index_col="ind")
raw_cvs

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
ind,,,,,,,,,,,,
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано
...,...,...,...,...,...,...,...,...,...,...,...,...
44739,"Мужчина , 30 лет , родился 17 января 1989",50000 руб.,"Финансист, аналитик, экономист, бухгалтер, мен...","Тверь , готов к переезду (Москва, Химки) , гот...",полная занятость,"полный день, удаленная работа","Опыт работы 7 лет 7 месяцев Финансист, аналит...","ООО ""IAS"" (независимый участник объединения Ru...",Руководитель субгруппы,Высшее образование 2015 Московский гуманитарн...,22.04.2019 12:32,Не указано
44740,"Мужчина , 27 лет , родился 5 марта 1992",39000 руб.,"Системный администратор, IT-специалист","Липецк , готов к переезду , готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день, ...","Опыт работы 7 лет Системный администратор, IT...",ИП Пестрецов,Предприниматель,Высшее образование (Бакалавр) 2016 Воронежски...,22.04.2019 13:11,Не указано
44741,"Женщина , 48 лет , родилась 26 декабря 1970",40000 руб.,"Аналитик данных, Математик","Челябинск , готова к переезду , готова к редки...",полная занятость,"полный день, удаленная работа",Опыт работы 21 год 5 месяцев Январь 1998 — по...,"ОАО «ЧМК», Исследовательско-Технологический Це...",Начальник группы аналитики,Высшее образование 2000 Южно-Уральский госуда...,09.04.2019 05:07,Не указано


In [17]:
raw_cvs.columns

Index(['Пол, возраст', 'ЗП', 'Ищет работу на должность:',
       'Город, переезд, командировки', 'Занятость', 'График', 'Опыт работы',
       'Последнее/нынешнее место работы', 'Последняя/нынешняя должность',
       'Образование и ВУЗ', 'Обновление резюме', 'Авто'],
      dtype='object')

In [18]:
raw_cvs['text'] = (
    raw_cvs['Пол, возраст'] + " . " +
    raw_cvs['Ищет работу на должность:'] + " . " +
    raw_cvs['Город, переезд, командировки'] + " . " +
    raw_cvs['Занятость'] + " . " +
    raw_cvs['График'] + " . " +
    raw_cvs['Опыт работы'] + " . " +
    raw_cvs['Последнее/нынешнее место работы'] + " . " +
    raw_cvs['Последняя/нынешняя должность'] + " . " +
    raw_cvs['Образование и ВУЗ']
).astype(str)
raw_cvs['text'][0]

'Мужчина ,  39 лет , родился 27 ноября 1979 . Системный администратор . Советск (Калининградская область) , не готов к переезду , не готов к командировкам . частичная занятость, проектная работа, полная занятость . гибкий график, полный день, сменный график, вахтовый метод, удаленная работа . Опыт работы 16 лет 10 месяцев  Август 2010 — по настоящее время 8 лет 10 месяцев МАОУ "СОШ № 1 г.Немана" Системный администратор Обслуживание ПК,установка ПО, ремонт, периферийной техники, Интернет локальная сеть. Ведение Электронного журнала, сайта организации.  Август 2002 — Август  2010 8 лет 1 месяц ТС "ВЕСТЕР-ИНФО" Старший продавец, директор отдела Продажи компьютерной техники . МАОУ "СОШ № 1 г.Немана" . Системный администратор . Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота, Калининград судоводительский, Организация и безопасность движения'

In [19]:
processed_cvs = raw_cvs[raw_cvs['text'].notnull()][['text']]
processed_cvs

,text
ind,
0,"Мужчина , 39 лет , родился 27 ноября 1979 . С..."
1,"Мужчина , 60 лет , родился 20 марта 1959 . Те..."
2,"Женщина , 36 лет , родилась 12 августа 1982 ...."
3,"Мужчина , 38 лет , родился 25 июня 1980 . Веб..."
4,"Женщина , 26 лет , родилась 3 марта 1993 . Ре..."
...,...
44739,"Мужчина , 30 лет , родился 17 января 1989 . Ф..."
44740,"Мужчина , 27 лет , родился 5 марта 1992 . Сис..."
44741,"Женщина , 48 лет , родилась 26 декабря 1970 ...."


In [20]:
to_csv(cvs_embeddings_path, processed_cvs.sample(20_000))

Total items: 20000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
